## Initialization

In [1]:
import sys
#sys.path.append('/Users/chandrasekharmukherjee/Home/Centrality framework/Modules')
sys.path.append('I:/내 드라이브/backup/document/USC/Research/MCPC/Code/Codes')

#Import packages
#imports
import time
import pandas as pd
from collections import Counter
import sknetwork


from sknetwork.ranking import PageRank
from sknetwork.ranking import Betweenness
from sknetwork.ranking import Closeness

#import umap as umap

from numba.typed import List
import warnings
from numba import njit
import pynndescent
import numpy as np
from sklearn.cluster import SpectralClustering
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib
import operator
from sklearn.utils.extmath import randomized_svd
from random import randint
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
import scipy
#from umap.umap_ import *
import math
from random import randint
import keras
from keras.datasets import mnist
from keras.datasets import cifar10
from keras.datasets import cifar100
from keras.datasets import fashion_mnist
import scanpy
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics import adjusted_mutual_info_score, roc_auc_score
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score

#import community as community_louvain
from sklearn.cluster import KMeans
from sklearn.metrics import fowlkes_mallows_score

import scipy

import igraph 
import networkx as nx


import metric as met 
import embedding as embed   
import data_utils_ch as data_util

%load_ext autoreload
%autoreload 2
import importlib

import metric as met 
import embedding as embed   
import data_utils_ch as data_util
from subgraph import *


met=importlib.reload(met)
embed=importlib.reload(embed)
data_util=importlib.reload(data_util)

## Testing every dataset with top k % nodes induced subgraph (Louvain)

### Top k% induced subgraph + Strong Majority Vote later

##### PDF Export version (Final Working Version)

In [27]:
import Cust_Top as Cust
import networkx.algorithms.community.louvain as Louv
from sklearn.metrics.cluster import normalized_mutual_info_score as NMI
from collections import deque
from matplotlib.backends.backend_pdf import PdfPages
import os
from subgraph import * 

res_=[] #0.26, 0.24 is best for Louvain, 3.3, 2.0
labels=[]
total_labels=[]
names = []

data_names = ['mRNA','miRNA','seeds','breast-cancer','Omniglot','bbc_news','20NewsGroups_tfdif','biorxiv','big_patent','Citeseer','Eu core','FashionMNIST','MNIST','Cora','Cora full']

# FR_ord = ['Rd', 'Ord']
# FR_Recalc = ['FR_avg', 'FR_Re']
FR_tp = ['FL','FL_ng','FL_ng_prop','PageRank']
Num_hops = {0:'5',1:'log(n)'}
Mod_tp = {0: 'Louv'}

#top k% nodes for induced subgraph
k=0.2
traverse = {0:'Reset',1:'Static'}
ord = {0:'Rand',1:'FR_order'}

#list of plotted figures
figures = []

for data in data_names: 
    pdf_name = './Plots/'+data+'/'+data+'.pdf'
    
    for survive in [0]:
        '''
        Get the Graph from Edge_List and get the labels
        '''
        #DO not recalculate Graph for non bulk RNA datasets
        if data not in ['mRNA','miRNA'] and survive==1:
            continue
        G, label= data_to_graph(data,survive)
        #add self loop to nodes without outgoing edges
        for node in G.nodes():
            if not list(G.successors(node)):
                G.add_edge(node, node, weight=1)
        print('Number of nodes:',G.number_of_nodes())


        data_directory = f'./Plots/{data}'
        file_path = f'{data_directory}/{data}.txt'
        os.makedirs(data_directory, exist_ok=True)
        with open(file_path, 'a') as f:
            f.write('------------------------------------------------------------------------\n')
        
        for trav in [0]:
            for node_ord in [1]:
                for Num_hop in [1]:
                    for res in [0.5,1,2]:
                        for Mod_type in [0]:
                            plt.figure() # figure for NMI
                            plt.figure() # figure for purity
                            plt.figure() # figure for Incoming Edge Degrees
                            for FR_type in range(4):
                                
                                

                                name =  data + ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops'
                                if FR_type==3:
                                    name = data + ' (top ' + str(k*100) + '%): PageRank'
                                if data in ['mRNA','miRNA']:
                                    name = data +  ' (top ' + str(k*100) + '%): '+ str(FR_tp[FR_type]) + ' + '+ str(Num_hops[Num_hop]) +' hops' + ' + Survive: '+str(survive)
                                    if FR_type==3:
                                        name = data + ' (top ' + str(k*100) + '%): PageRank' + ' + Survive: '+str(survive)
                                print(name)
                                
                                # Create the directory if it doesn't exist
                                os.makedirs(data_directory, exist_ok=True)
                                with open(file_path, 'a') as f:
                                    f.write(name+'\n')
                                
                                '''
                                Get induced subgraph of top k% nodes in FR value
                                '''
                                #Get induced subgraph
                                if Num_hop==0:
                                    node2FR = calc_FlowRank(G, FR_type, 5)
                                else:
                                    node2FR = calc_FlowRank(G, FR_type, np.log2(G.number_of_nodes()))
                                #Get top k% nodes
                                node_list = sorted(G.nodes, key=lambda x: node2FR[x], reverse=True)
                                H, node_ordered_by_FR = getInducedSubgraph(G, k, node_list) # H = induced subgraph



                                '''
                                Calculate NMI and Purity for different resolutions
                                '''
                                
                                # partition = RC10.louvain_partitions(H, seed=0,resolution=res, FR_type=FR_type, Mod_type=Mod_type)
                                partition = Louv.louvain_partitions(H, seed=0,resolution=res )
                                partition_ = deque(partition, maxlen=1).pop()
                                '''Run Louvain Again on the rest ( Warm Start )'''
                                # k_ = int(k*len(node_list))
                                # nodes_rest = node_list[k_:]
                                # for node in nodes_rest:
                                #     partition_.append({node})
                                # label_ = part_to_full_label(partition_,G.number_of_nodes())
                                # part_ = Cust.louvain_partitions(G, seed=0,resolution=res,init_part=partition_)
                                # part = deque(part_, maxlen=1).pop()
                                # label_ = part_to_full_label(part,G.number_of_nodes())


                                '''
                                Add nodes to the subgraph by strong majority vote
                                '''
                                #See the trend by adding node to the subgraph by strong majority vote (in order of FlowRank Value)
                                H_label = part_to_full_label(partition_,G.number_of_nodes())
                                #print('H_label: ',H_label, 'len: ',len(H_label))
                                

                                

                                NMI_List, Purity_List, InEdge_List = merge_by_vote(trav, node_ord, node_ordered_by_FR,H_label, G, label)

                                
                                #Plot NMI_List
                                fig_numbers = plt.get_fignums()
                                nmi_fig =fig_numbers[-1]
                                purity_fig = fig_numbers[-2]
                                InEdge_fig = fig_numbers[-3]

                                #plot NMI
                                plt.figure(nmi_fig)
                                plt.plot(NMI_List, label = str(FR_tp[FR_type]))
                                with open(file_path, 'a') as f:
                                    f.write('# of nodes not included: ' + str(G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) + '\n')
                                    f.write('NMI after top ' + str(k*100) + '% nodes: ' + str(round(NMI_List[0],3)) + '\n')
                                    f.write('Max NMI: ' + str(round(max(NMI_List),3)) + '\n')
                                print('Number of nodes not included: ' , G.number_of_nodes() - H.number_of_nodes() - len(NMI_List)+1) 
                                print('NMI after top ', k*100, '% nodes:', round(NMI_List[0],3))
                                print('Max NMI: ', round(max(NMI_List),3))

                                #plot Purity
                                plt.figure(purity_fig)
                                plt.plot(Purity_List, label = str(FR_tp[FR_type]))
                                with open(file_path, 'a') as f:
                                    f.write('Purity after top ' + str(k*100) + '% nodes: ' + str(round(Purity_List[0],3)) + '\n')
                                    f.write('Max Purity: ' + str(round(max(Purity_List),3)) + '\n')

                                #plot InEdge Degrees
                                plt.figure(InEdge_fig)
                                plt.plot(InEdge_List, label = str(FR_tp[FR_type]))
                            
                                
                        plt.figure(nmi_fig)    
                        #Plot all at once
                        plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                        plt.ylabel('NMI')
                        plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|'
                                +str(traverse[trav]) + ' | ' + str(ord[node_ord]))
                        plt.legend()

                        plt.figure(purity_fig)
                        #Plot all at once
                        plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                        plt.ylabel('Purity')
                        plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|'
                                +str(traverse[trav]) + ' | ' + str(ord[node_ord]))
                        plt.legend()

                        plt.figure(InEdge_fig)
                        #Plot all at once
                        plt.xlabel('Number of nodes added after top ' + str(k*100) + '% nodes')
                        plt.ylabel('InEdge Degrees')
                        plt.title(data + ' | top ' + str(k*100) + '%' + '| Num_hops: ' + str(Num_hops[Num_hop]) + '|res: ' + str(res)+ '|'
                                +str(traverse[trav]) + ' | ' + str(ord[node_ord]))
                        plt.legend()
    
    p = PdfPages(pdf_name)            
    fig_nums = plt.get_fignums() 
    figs = [plt.figure(n) for n in fig_nums] 
	
	# iterating over the numbers in list 
    for fig in figs: 
		# and saving the files 
        fig.savefig(p, format='pdf') 
	
	# close the object 
    p.close() 
    plt.close('all')
    
        
            

                                
                    

                    
                    



[0 1 2 3 4 5 6 7 8 9]


KeyboardInterrupt: 